# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
sqrt_value_sum = 0
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    #print(sqrt_value)
    sqrt_value_sum = sqrt_value_sum+sqrt_value

print(sqrt_value_sum)


8.382332347441762


In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [5]:
from itertools import chain

age_sum = 0
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

#for person in people_1():
#    print(person)

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


#for person in people_2():
#    person_2 = person | person_2

person_3 = chain(people_1(), people_2())
for item in person_3:
    print(item)
    age_sum = age_sum + item['Age']

print(f"Total Age: {age_sum}")


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Total Age: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
%%capture
pip install dlt[duckdb]

In [ ]:
%%capture
pip install numpy

In [ ]:
%%capture
pip install pandas

In [6]:
import dlt
import duckdb

# define the connection to load to.
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_list_2')

person_data = []
for people in people_1():
    person_data.append(people)

print(person_data) 

# run with merge write disposition.
info = pipeline.run(person_data,table_name="people_table_2",write_disposition="merge",primary_key="ID")
print(info)
print("============== Load first table complete =================")

person_data = []
for people in people_2():
    person_data.append(people)

print(person_data) 

# run with merge write disposition.
info = pipeline.run(person_data,table_name="people_table_2",write_disposition="merge",primary_key="ID")
print(info)
print("============== Load second table complete =================")

# show the outcome
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
#print('Loaded tables: ')
#display(conn.sql("show tables"))

people_df = conn.sql("SELECT * FROM people_table_2").df()
display(people_df)

display(conn.sql("SELECT sum(age) FROM people_table_2"))


[{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}, {'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}, {'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}, {'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}, {'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}]
Pipeline dlt_ipykernel_launcher load step completed in 0.67 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_list_2
The duckdb destination used duckdb:///D:\Automations_work\dataeng_zoomcamp_2024\dlt\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708404866.073966 is LOADED and contains no failed jobs
============== Load first table complete =================
[{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}, {'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}, {'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}, {'ID': 6, 'Name': '

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708404866.073966,ozYjX7/QGYGteg,None
1,2,Person_2,27,City_A,1708404866.073966,fWom+bNlRZh40g,None
2,7,Person_7,37,City_B,1708404867.1320925,LvIUayXKlSPAyQ,Job_7
3,5,Person_5,35,City_B,1708404867.1320925,9P6G3gHOEEob0A,Job_5
4,4,Person_4,34,City_B,1708404867.1320925,Rxg5BFo/+AiOTA,Job_4
5,3,Person_3,33,City_B,1708404867.1320925,XD2FGppUUHLiyQ,Job_3
6,6,Person_6,36,City_B,1708404867.1320925,Qkd7xKotPBiaRQ,Job_6
7,8,Person_8,38,City_B,1708404867.1320925,cAaMNaRZF0q41A,Job_8


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

In [ ]:
import dlt
import duckdb

# define the connection to load to.
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_list')

person_data = []
for people in people_1():
    person_data.append(people)

print(person_data) 

# run with merge write disposition.
info = pipeline.run(person_data,table_name="people_table_1",write_disposition="merge",primary_key="ID")

print(info)

person_data = []
for people in people_2():
    person_data.append(people)

print(person_data) 

# run with merge write disposition.
info = pipeline.run(person_data,table_name="people_table_1",write_disposition="merge",primary_key="ID")


# show the outcome
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people_df = conn.sql("SELECT * FROM people_table_1").df()
display(people_df)


In [ ]:
display(conn.sql("SELECT sum(age) as age_sum FROM people_table"))

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX